### Train an classification model

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO
import verifyml.model_card_toolkit as mctlib
import verifyml.model_tests.utils as utils


from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

sns.set_theme(style="whitegrid")

In [ ]:
# Load training dataset
df_train = pd.read_csv('./dataset.csv')
X_train_full = df_train.iloc[:,1:].drop('TRS', axis=1)


y_train_full = df_train['TRS']

# Load evaluation dataset
df_eval = pd.read_csv('./trs_eval.csv')
X_eval = df_eval.iloc[:,1:]